In [10]:
# Lots of imports needed
import pandas as pd
#import numpy as np
import geopandas as gpd
import json
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool, CustomJS
from bokeh.layouts import widgetbox, row, column
from bokeh.models.widgets import Dropdown

In [3]:
selectedData = pd.read_csv('../Data/Nigeria/filtered_nigeria_data.csv')

In [6]:
# loading in geojson
shapefile = '../Data/Nigeria/Nigeria_Shape/AFRO_IUs_201812.shp'
gdf = gpd.read_file(shapefile)
nigeria = gdf[gdf.ADMIN0 == 'Nigeria'].copy()

In [7]:
# merging the two data sets
merged = nigeria.merge(selectedData, left_on='IUs_NAME', right_on='district_names')
# summing up all the requested columns and dividing them by the target population
merged['alb_mbd_ratio'] = (merged['lf_alb_total_procured'] + merged['sac_alb_total_procured'] + merged['sth_mbd_total_procured']) / merged['total_treatment_alb_mbd']
merged['mbd_ratio'] = (merged['sth_mbd_total_procured']) / merged['total_target_mbd']
merged['pzq_ratio'] = merged['pzq_total_procured'] / merged['pzq_total_treatment']
merged['ivm_ratio'] = merged['ivm_total_procured'] / merged['ivm_total_treatment']

In [8]:
merged.loc[(pd.isna(merged.pzq_ratio)), 'pzq_ratio'] = 0
merged.loc[(pd.isna(merged.mbd_ratio)), 'mbd_ratio'] = 0
merged.loc[(pd.isna(merged.alb_mbd_ratio)), 'alb_mbd_ratio'] = 0
merged.loc[(pd.isna(merged.ivm_ratio)), 'ivm_ratio'] = 0
merged.loc[(merged.sth_mbd_total_procured == 0) & (merged.total_target_mbd != 0), 'mbd_ratio'] = None



In [35]:
def changeDist(dist):
    if dist == 'all':
        new_data = merged
    else:
        new_data = merged[merged.IUs_NAME == dist]
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data

geosource = GeoJSONDataSource(geojson = changeDist('all'))
palette = brewer['YlGnBu'][8]
palette = palette[::-1]
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 2.5, nan_color='#fa0505')

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal')

hover = HoverTool(tooltips = [ ('District','@IUs_NAME'),('ratio', '@mbd_ratio')])

p = figure(title = 'Test', plot_height = 600 , plot_width = 950, toolbar_location = None, tools=[hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.patches('xs','ys', source = geosource,fill_color = {'field' :'mbd_ratio', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
p.add_layout(color_bar, 'below')

def update_plot(attr, old, new):
    ctry = slider.value
    #new_data = changeDist(ctry)
    geosource = new_data
    p.title.text = 'MBD Ratio, %d' %ctry
menu = []
for dist_name in merged.IUs_NAME:
    menu.append((dist_name, dist_name))
    
    
###def callback(source=geosource, code="""
###    var data = source.data;
#    var f = cb_obj.value
#    var index = data.IUs_NAME.indexOf(f)
#    console.log(JSON.parse(data))
#    source.change.emit();
#""")
    
#slider = Slider(title = 'Year',start = 1975, end = 2016, step = 1, value = 2016)
#slider.on_change('value', update_plot)
    
dropdown = Dropdown(label="Dropdown button", button_type="warning", menu=menu)
#dropdown.on_change('value', update_plot)
dropdown.js_on_change('value', CustomJS(update_plot))
#layout = column(p,widgetbox(slider))
#curdoc().add_root(layout)
layout = column(dropdown, p)
#curdoc().add_root(layout)

#output_notebook()
show(layout)
#show(dropdown)

TypeError: __init__() takes 1 positional argument but 2 were given